In [1]:
import pandas as pd

# How to export CloudFit Azure RateCard

Run the kusto query:
```
CarbonEmissionResourceCard_Snapshot |
where Timestamp == '2022-08-01T00:00:00Z' |
join kind=inner MeterMapping_Materialized_Snapshot on MeterGuid |
summarize arg_max(Timestamp,*) by MeterGuid
```

Then export to CSV

In [2]:
az_co2_ratecard = pd.read_csv('./cloudfit/export.csv')

# CloudFit Azure RateCard

These units are units depending on the following classification

| Resource | Common Unit | Definition
|:--:|:--:|:--:|
| Storage | Gb-hour | One GB stored for one hour in Azure
| Compute | Core-hour | One core used for one hour based on the size of VM
| Network | GB | One GB of data transmitted 

In [5]:
az_co2_ratecard[az_co2_ratecard.MeterCategoryName == 'Virtual Machines']

,Group,MeterGuid,Timestamp,EmissionsPerUnitResourceUsage,EmissionsPerUnit_Scope1_mtCO2e_Azure,EmissionsPerUnit_Scope2_location_mtCO2e_Azure,EmissionsPerUnit_Scope2_market_mtCO2e_Azure,EmissionsPerUnit_Scope3_mtCo2e,MeterGuid1,MeterCategoryName,MeterSubcategoryName,MeterName,MeterRegionName
11516,NaN,039d49e6-c713-5929-a310-a9a53a018338,2022-08-01T00:00:00Z,2.583586e-04,3.404295e-07,4.922137e-06,0.000000,2.530960e-04,039d49e6-c713-5929-a310-a9a53a018338,Virtual Machines,Esv4 Series,E32s v4,SE Central
11517,NaN,04de6b3c-2dad-548b-9a78-5edf5cd19652,2022-08-01T00:00:00Z,4.120700e-05,5.487986e-08,2.382574e-05,0.000000,1.732637e-05,04de6b3c-2dad-548b-9a78-5edf5cd19652,Virtual Machines,NCasv3 T4 Series Windows,NC4as T4 v3,US Central
11518,NaN,058535f3-e927-4143-9b39-4e9ec2bcc825,2022-08-01T00:00:00Z,5.956743e-06,1.466834e-08,1.824460e-06,0.000000,4.117614e-06,058535f3-e927-4143-9b39-4e9ec2bcc825,Virtual Machines,D/DS Series,D11/DS11,US East
11519,NaN,081e60d3-d4d3-5b99-b839-587426650fd4,2022-08-01T00:00:00Z,7.196766e-05,9.656117e-07,2.242306e-05,0.000000,4.857899e-05,081e60d3-d4d3-5b99-b839-587426650fd4,Virtual Machines,Esv5 Series Windows,E48s v5 Low Priority,EU North
11520,NaN,08898009-bd5e-48c7-a926-1029e815fdc3,2022-08-01T00:00:00Z,1.242274e-06,5.704752e-09,6.344647e-07,0.000000,6.021049e-07,08898009-bd5e-48c7-a926-1029e815fdc3,Virtual Machines,Av2 Series,A2 v2 Low Priority,US South Central
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45325,NaN,ee6e2e65-b1d2-5863-a209-c584a58606a8,2022-08-01T00:00:00Z,3.418159e-06,3.901330e-09,1.182389e-06,0.000000,2.231868e-06,ee6e2e65-b1d2-5863-a209-c584a58606a8,Virtual Machines,Dsv4 Series,D2s v4,EU West
45326,NaN,f374ce73-912f-5a93-8fd2-d4a5986b4a52,2022-08-01T00:00:00Z,1.444544e-05,1.648736e-08,4.996881e-06,0.000000,9.432070e-06,f374ce73-912f-5a93-8fd2-d4a5986b4a52,Virtual Machines,Esv4 Series,E32-16s v4 Low Priority,EU West
45327,NaN,f69b71b2-4d12-406d-82f7-11903f972e6c,2022-08-01T00:00:00Z,1.706866e-05,3.556164e-08,8.988406e-06,0.000002,8.044690e-06,f69b71b2-4d12-406d-82f7-11903f972e6c,Virtual Machines,Av2 Series Windows,A4m v2,AU East
45328,NaN,fa752888-e8be-54d7-9723-4b44747350aa,2022-08-01T00:00:00Z,3.793761e-05,8.124693e-09,1.450433e-05,0.000000,2.342516e-05,fa752888-e8be-54d7-9723-4b44747350aa,Virtual Machines,D/DS Flex Series,D8/D8s,US West 3


# Constants

In [6]:
hrs_per_month = 730

# Functions to retrieve each component's (compute, network, storage) CO2 emissions

In [29]:
def get_emissions_for_vm_sku_in_kg(vm_sku, region='AU East'):
    return az_co2_ratecard[(az_co2_ratecard.MeterCategoryName == 'Virtual Machines') & (az_co2_ratecard.MeterName == vm_sku) & (az_co2_ratecard.MeterRegionName == region)].iloc[0].EmissionsPerUnitResourceUsage * hrs_per_month * 1000

In [30]:
def get_emissions_for_vm_disk_in_kg(disk_sku, region='AU East'):
    return az_co2_ratecard[(az_co2_ratecard.MeterCategoryName == 'Storage') & (az_co2_ratecard.MeterName == disk_sku) & (az_co2_ratecard.MeterRegionName == region)].iloc[0].EmissionsPerUnitResourceUsage * 1000

In [60]:
def get_emissions_for_vm_network(data_out_in_gb, region='All'):
    return (az_co2_ratecard[(az_co2_ratecard.MeterCategoryName == 'Virtual Network') & (az_co2_ratecard.MeterName == 'Inter-Region Egress') & (az_co2_ratecard.MeterRegionName == region)].iloc[0].EmissionsPerUnitResourceUsage + az_co2_ratecard[(az_co2_ratecard.MeterCategoryName == 'Virtual Network') & (az_co2_ratecard.MeterName == 'Inter-Region Ingress') & (az_co2_ratecard.MeterRegionName == region)].iloc[0].EmissionsPerUnitResourceUsage) * data_out_in_gb * 1000 

In [63]:
def get_emissions_for_vm_per_month(vm_sku, disk_sku, data_out_in_gb, region='AU East'):
    return get_emissions_for_vm_sku_in_kg(vm_sku, region) + get_emissions_for_vm_disk_in_kg(disk_sku, region) + get_emissions_for_vm_network(data_out_in_gb, 'All')

# Exercise functions

In [66]:
d2_v2_emissions =  get_emissions_for_vm_per_month('D2 v2/DS2 v2', 'S10 Disks', 50)
print(f'VM: {d2_v2_emissions} kg in CO2Eq Emissions/Month')

VM: 7.899807158759507 kg in CO2Eq Emissions/Month


In [67]:
d3_v2_emissions =  get_emissions_for_vm_per_month('D3 v2/DS3 v2', 'S10 Disks', 50)
print(f'VM: {d3_v2_emissions} kg in CO2Eq Emissions/Month')

VM: 15.302119696967235 kg in CO2Eq Emissions/Month


In [69]:
f2s_v2_emissions =  get_emissions_for_vm_per_month('F2s v2', 'S10 Disks', 50)
print(f'VM: {f2s_v2_emissions} kg in CO2Eq Emissions/Month')

VM: 5.845072717548771 kg in CO2Eq Emissions/Month


In [70]:
e2_v4_emissions =  get_emissions_for_vm_per_month('E2s v4', 'S10 Disks', 50)
print(f'VM: {e2_v4_emissions} kg in CO2Eq Emissions/Month')

VM: 6.898782687400419 kg in CO2Eq Emissions/Month


# AKS - Related Emissions

In [53]:
def get_emissions_for_aks_in_kg():
    return az_co2_ratecard[(az_co2_ratecard.MeterCategoryName == 'Azure Kubernetes Service') & (az_co2_ratecard.MeterName == 'Standard Uptime SLA')].iloc[0].EmissionsPerUnitResourceUsage * hrs_per_month * 1000

In [54]:
aks = get_emissions_for_aks_in_kg()

In [55]:
aks

3.2851987288340494